In [ ]:
# import libraries
import snowflake.connector
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from decimal import Decimal, ROUND_CEILING

In [ ]:
# connect to snowflake
conn = snowflake.connector.connect(
user='Vance',
password='',
role = 'ANALYTICS_TEAM',
account= 'vl37350',
warehouse = 'READONLY_COMPUTE',
database = 'ANALYTICS',
schema = 'CSV'
)

In [ ]:
# past campaign data
cur1 = conn.cursor()
c = '''SELECT ID,
CHANNEL,
TYPE,
QUANTITY,
LIST_FEE,
POSTAGE_FEE,
MAIL_DATE,
RESPONSE_CURVE_ID,
CAMPAIGN_MONTH FROM FIVETRAN_DB.APPLYDB_PUBLIC.CAMPAIGNS'''
try:
    cur1.execute(c)
    all_rows = cur1.fetch_pandas_all()
    field_names = [i[0] for i in cur1.description]
finally:
    cur1.close()
campaigns = pd.DataFrame(all_rows)
campaigns.columns = field_names

# response curves
cur2 = conn.cursor()
rc = '''SELECT TYPE, DAYSSINCEDROP, SUM(RESPONSE_CURVE) AS CURVE FROM (
SELECT TYPE,DAYSSINCEDROP,(RESPONSES/SUM(RESPONSES) OVER (PARTITION BY TYPE)) AS RESPONSE_CURVE FROM (

SELECT TYPE, ACTIVATION_METHOD, TO_DATE(RESPONSE_DATE) - TO_DATE(MAIL_DATE)  AS DAYSSINCEDROP, COUNT(RESPONSE_ID) AS RESPONSES FROM (
SELECT *
FROM (SELECT *,CONCAT(20,SUBSTR(ACTIVATION_CODE,6,2),'-',SUBSTR(ACTIVATION_CODE,2,2)) AS "MONTH_OF_MAIL" FROM "FIVETRAN_DB"."APPLYDB_PUBLIC"."ACTIVATION_CODES"
WHERE (THIRD_PARTY_RECORD_ID IS NULL OR try_to_numeric(THIRD_PARTY_RECORD_ID) > 0) AND _fivetran_deleted= 'false' ) A

LEFT JOIN (SELECT *,CURRENT_DATE() - MAIL_DATE AS "DAYS_SINCE_DROP"
                    ,CASE WHEN "DAYS_SINCE_DROP" > 55 THEN 55 ELSE "DAYS_SINCE_DROP" END AS    "DAYS_SINCE_DROP_TRT"
           FROM "FIVETRAN_DB"."APPLYDB_PUBLIC"."CAMPAIGNS") C
           ON C.ID = A.CAMPAIGN_ID
LEFT JOIN  (SELECT  ID AS RESPONSE_ID, ACTIVATION_METHOD,ACTIVATION_CODE_ID,CREATED_AT as RESPONSE_DATE, ROW_NUMBER()OVER (PARTITION BY ACTIVATION_CODE_ID ORDER BY RESPONSE_ID) RR
            FROM "FIVETRAN_DB"."APPLYDB_PUBLIC"."RESPONSES") R
            ON R.ACTIVATION_CODE_ID = A.ID AND  R.RR = 1
WHERE CURRENT_DATE() - MAIL_DATE  BETWEEN 60 AND 240
) GROUP BY 1,2,3 having DAYSSINCEDROP <= 60 ORDER BY 1,2,3
) ORDER BY 1,2
) GROUP BY 1,2 ORDER BY 1,2'''
try:
    cur2.execute(rc)
    all_rows = cur2.fetch_pandas_all()
    field_names = [i[0] for i in cur2.description]
finally:
    cur2.close()
rcurves = pd.DataFrame(all_rows)
rcurves.columns = field_names

# response/sub numbers for campaign types
cur3 = conn.cursor()
ib_ss_responses = '''SELECT
TYPE, ACTIVATION_METHOD, COUNT(RESPONSE_ID) AS RESPONSES, COUNT(SUB_DATE) AS SUBS
FROM (SELECT * FROM (SELECT *,CONCAT(20,SUBSTR(ACTIVATION_CODE,6,2),'-',SUBSTR(ACTIVATION_CODE,2,2)) as "MONTH_OF_MAIL" FROM "FIVETRAN_DB"."APPLYDB_PUBLIC"."ACTIVATION_CODES" WHERE (THIRD_PARTY_RECORD_ID IS NULL OR try_to_numeric(THIRD_PARTY_RECORD_ID) > 0) AND _fivetran_deleted= 'false') A

LEFT JOIN (SELECT *,CURRENT_DATE() - MAIL_DATE AS "DAYS_SINCE_DROP"
                    ,CASE WHEN "DAYS_SINCE_DROP" > 55 THEN 55 ELSE "DAYS_SINCE_DROP" END AS    "DAYS_SINCE_DROP_TRT"
           FROM "FIVETRAN_DB"."APPLYDB_PUBLIC"."CAMPAIGNS") C
           ON C.ID = A.CAMPAIGN_ID     -- join on campaign id

LEFT JOIN  (SELECT ID AS RESPONSE_ID, ACTIVATION_METHOD,ACTIVATION_CODE_ID,CREATED_AT as RESPONSE_DATE, ROW_NUMBER()OVER (PARTITION BY ACTIVATION_CODE_ID ORDER BY RESPONSE_ID) RR
            FROM "FIVETRAN_DB"."APPLYDB_PUBLIC"."RESPONSES") R
            ON R.ACTIVATION_CODE_ID = A.ID AND  R.RR = 1

LEFT JOIN (SELECT PROPERTY_DM_ACCT_NUM_C, PROPERTY_APP_SUBMITTED_DATE_TIME AS SUB_DATE
            FROM "FIVETRAN_DB"."HUBSPOT"."DEAL") D
            ON TRIM(A.ACTIVATION_CODE) = D.PROPERTY_DM_ACCT_NUM_C
WHERE "MAIL_DATE" >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL '2 months')
) GROUP BY 1,2 HAVING ACTIVATION_METHOD IS NOT NULL'''
try:
    cur3.execute(ib_ss_responses)
    all_rows = cur3.fetch_pandas_all()
    field_names = [i[0] for i in cur3.description]
finally:
    cur3.close()
campaign_rates = pd.DataFrame(all_rows)
campaign_rates.columns = field_names

conn.close()

In [ ]:
# inputs (response rates, campaigns for forecast month)
response_rates = pd.read_csv('ResponseRateInput.csv')
new_camps = pd.read_excel('NewCampaignsInput.xlsx')
# ib call assumption
call_to_sub = Decimal(str(.30))
# twilio preapproved
twilio = Decimal(str(.50))
# twilio abandons
ab_ss = Decimal(str(.86))
ss_bizhrs = Decimal(str(.85))
# start and end date
start = '2024-09-01'
end = '2024-09-30'

In [ ]:
# reformat response/sub rate table
DMA = campaign_rates[campaign_rates['TYPE'] == 'DMA']
DMR = campaign_rates[campaign_rates['TYPE'] == 'DMR']
DMO = campaign_rates[campaign_rates['TYPE'] == 'DMO']
DMC = campaign_rates[campaign_rates['TYPE'] == 'DMC']
t = pd.merge(DMA, DMR, on='ACTIVATION_METHOD', how='outer')
x = pd.merge(DMC, DMO, on='ACTIVATION_METHOD', how='outer')
rates = pd.merge(t, x, on='ACTIVATION_METHOD', how='outer')
rates = rates.rename(columns={
    'RESPONSES_x_x': 'DMA_RESPONSES',
    'SUBS_x_x': 'DMA_SUBS',
    'RESPONSES_y_x': 'DMR_RESPONSES',
    'SUBS_y_x': 'DMR_SUBS',
    'RESPONSES_x_y': 'DMC_RESPONSES',
    'SUBS_x_y': 'DMC_SUBS',
    'RESPONSES_y_y': 'DMO_RESPONSES',
    'SUBS_y_y': 'DMO_SUBS'
})
rates = rates[['ACTIVATION_METHOD',
              'DMA_RESPONSES', 'DMA_SUBS',
              'DMR_RESPONSES', 'DMR_SUBS',
              'DMC_RESPONSES', 'DMC_SUBS',
              'DMO_RESPONSES', 'DMO_SUBS']]

DMA_rate = rates[['ACTIVATION_METHOD', 'DMA_RESPONSES', 'DMA_SUBS']]
DMR_rate = rates[['ACTIVATION_METHOD', 'DMR_RESPONSES', 'DMR_SUBS']]
DMC_rate = rates[['ACTIVATION_METHOD', 'DMC_RESPONSES', 'DMC_SUBS']]
DMO_rate = rates[['ACTIVATION_METHOD', 'DMO_RESPONSES', 'DMO_SUBS']]

# calculate ib and ss sub rate
DMA_rate.loc[len(DMA_rate)] = {'ACTIVATION_METHOD': 'total',
                               'DMA_RESPONSES': DMA_rate['DMA_RESPONSES'].sum(),
                               'DMA_SUBS': DMA_rate['DMA_SUBS'].sum()}
DMA_rate.loc[:, 'RATE'] = DMA_rate['DMA_SUBS'] / DMA_rate['DMA_RESPONSES']
DMA_rate['ADJUSTED_RATE'] = DMA_rate['RATE']/DMA_rate.loc[2, 'RATE']

DMR_rate.loc[len(DMR_rate)] = {'ACTIVATION_METHOD': 'total',
                               'DMR_RESPONSES': DMR_rate['DMR_RESPONSES'].sum(),
                               'DMR_SUBS': DMR_rate['DMR_SUBS'].sum()}
DMR_rate.loc[:, 'RATE'] = DMR_rate['DMR_SUBS']/DMR_rate['DMR_RESPONSES']
DMR_rate['ADJUSTED_RATE'] = DMR_rate['RATE']/DMR_rate.loc[2, 'RATE']

DMC_rate.loc[len(DMC_rate)] = {'ACTIVATION_METHOD': 'total',
                               'DMC_RESPONSES': DMC_rate['DMC_RESPONSES'].sum(),
                               'DMC_SUBS': DMC_rate['DMC_SUBS'].sum()}
DMC_rate.loc[:, 'RATE'] = DMC_rate['DMC_SUBS']/DMC_rate['DMC_RESPONSES']
DMC_rate['ADJUSTED_RATE'] = DMC_rate['RATE']/DMC_rate.loc[2, 'RATE']

DMO_rate.loc[len(DMO_rate)] = {'ACTIVATION_METHOD': 'total',
                               'DMO_RESPONSES': DMO_rate['DMO_RESPONSES'].sum(),
                               'DMO_SUBS': DMO_rate['DMO_SUBS'].sum()}
DMO_rate.loc[:, 'RATE'] = DMO_rate['DMO_SUBS']/DMO_rate['DMO_RESPONSES']
DMO_rate['ADJUSTED_RATE'] = DMO_rate['RATE']/DMO_rate.loc[2, 'RATE']
DMA_rate

C:\Users\Vance.Liu\AppData\Local\Temp\ipykernel_16768\962237092.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DMA_rate.loc[len(DMA_rate)] = {'ACTIVATION_METHOD': 'total',
C:\Users\Vance.Liu\AppData\Local\Temp\ipykernel_16768\962237092.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DMA_rate.loc[:, 'RATE'] = DMA_rate['DMA_SUBS'] / DMA_rate['DMA_RESPONSES']
C:\Users\Vance.Liu\AppData\Local\Temp\ipykernel_16768\962237092.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,ACTIVATION_METHOD,DMA_RESPONSES,DMA_SUBS,RATE,ADJUSTED_RATE
0,inbound,256,146,0.570312,0.847788
1,self-service,1510,1042,0.690066,1.025806
2,total,1766,1188,0.672707,1.000000


In [ ]:
# create columns for ib sub rate and ss sub rate in the new campaigns dataframe
dict = {
    'DMA': DMA_rate,
    'DMO': DMO_rate,
    'DMR': DMR_rate,
    'DMC': DMC_rate
}
for i in new_camps.index:
    new_camps.loc[i, 'IB_SUB_RATE'] = new_camps.loc[i, 'SUB_RATE']*dict[new_camps.loc[i, 'TYPE']].loc[0, 'ADJUSTED_RATE']
    new_camps.loc[i, 'SS_SUB_RATE'] = new_camps.loc[i, 'SUB_RATE']*dict[new_camps.loc[i, 'TYPE']].loc[1, 'ADJUSTED_RATE']
new_camps = (new_camps.drop(['SUB_RATE'], axis=1)).sort_values('MAIL_DATE')
new_camps

,TYPE,MAIL_DATE,QUANTITY,RESPONSE_RATE,PRELIM_RATE,IB_SUB_RATE,SS_SUB_RATE
0,DMA,2024-09-06,140000,0.8,40.81,57.649569,69.754775
5,DMR,2024-09-09,5924,2.6,30.00,75.408386,69.384468
9,DMO,2024-09-09,17526,4.0,47.00,89.067584,86.743312
1,DMA,2024-09-13,65000,0.8,40.81,57.649569,69.754775
6,DMR,2024-09-16,5173,2.6,30.00,75.408386,69.384468
10,DMO,2024-09-16,20592,4.0,47.00,89.067584,86.743312
2,DMA,2024-09-20,65000,0.8,40.81,57.649569,69.754775
7,DMR,2024-09-23,13743,2.6,30.00,75.408386,69.384468
11,DMO,2024-09-23,20138,4.0,47.00,89.067584,86.743312
3,DMA,2024-09-27,65000,0.8,40.81,57.649569,69.754775


In [ ]:
# inbound response rate
ib_rate = pd.DataFrame({'DMA': [DMA_rate.loc[0, 'DMA_RESPONSES']/DMA_rate.loc[2, 'DMA_RESPONSES']],
                        'DMO': [DMO_rate.loc[0, 'DMO_RESPONSES']/DMO_rate.loc[2, 'DMO_RESPONSES']],
                        'DMR': [DMR_rate.loc[0, 'DMR_RESPONSES']/DMR_rate.loc[2, 'DMR_RESPONSES']],
                        'DMC': [DMC_rate.loc[0, 'DMC_RESPONSES']/DMC_rate.loc[2, 'DMC_RESPONSES']]
                        })
ib_rate

,DMA,DMO,DMR,DMC
0,0.14496,0.110438,0.102181,0.081481


In [ ]:
# get current month and last two months
now = datetime.now()
now = now.replace(day=1)
next = now + relativedelta(months=1)
t = next + relativedelta(months=1)
cutoff = next - pd.Timedelta(days=80)
l = (now - relativedelta(months=1)).strftime('%Y-%m')
ll = (now - relativedelta(months=2)).strftime('%Y-%m')

In [ ]:
# create dataframe of past campaigns and add new campaigns
relevant_campaigns = campaigns[pd.to_datetime(campaigns['MAIL_DATE']) > cutoff]
relevant_campaigns = (relevant_campaigns
                      .groupby(['MAIL_DATE', 'TYPE'], as_index=False)
                      .agg({'QUANTITY': 'sum'})
                      .sort_values(by=['MAIL_DATE', 'TYPE'])
                      )
# merge relevant campaigns and response rates on month
relevant_campaigns['MAIL_DATE'] = pd.to_datetime(relevant_campaigns['MAIL_DATE'])
relevant_campaigns['MONTH'] = relevant_campaigns['MAIL_DATE'].dt.month
relevant_campaigns = pd.merge(relevant_campaigns, response_rates, on=['MONTH', 'TYPE'], how='left')
relevant_campaigns = relevant_campaigns.drop('MONTH', axis=1)


unique_rows = new_camps.merge(relevant_campaigns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
relevant_campaigns = pd.concat([relevant_campaigns, unique_rows], ignore_index=True)
relevant_campaigns = relevant_campaigns.drop_duplicates(subset=['MAIL_DATE', 'TYPE'], keep='first')
relevant_campaigns = relevant_campaigns.reset_index(drop=True)
relevant_campaigns = relevant_campaigns.fillna(0)
relevant_campaigns['PROJECTED_RESPONSES'] = (relevant_campaigns['RESPONSE_RATE']/100*relevant_campaigns['QUANTITY']).astype(int)
relevant_campaigns

,MAIL_DATE,TYPE,QUANTITY,RESPONSE_RATE,IB_SUB_RATE,SS_SUB_RATE,PRELIM_RATE,PROJECTED_RESPONSES
0,2024-07-15,DMO,16278,4.50,85.702838,79.791950,36.000000,732
1,2024-07-15,DMR,3609,2.37,69.961076,60.162845,38.000000,85
2,2024-07-19,DMA,77362,0.48,67.256839,64.014791,50.162866,371
3,2024-07-22,DMC,1800,0.60,65.000000,65.000000,84.000000,10
4,2024-07-22,DMO,24309,4.50,85.702838,79.791950,36.000000,1093
5,2024-07-22,DMR,4705,2.37,69.961076,60.162845,38.000000,111
6,2024-07-26,DMA,77362,0.48,67.256839,64.014791,50.162866,371
7,2024-07-29,DMO,11904,4.50,85.702838,79.791950,36.000000,535
8,2024-07-29,DMR,4619,2.37,69.961076,60.162845,38.000000,109
9,2024-08-02,DMA,60000,0.52,67.507279,62.454097,49.670000,312


In [ ]:
# Reformat response curve dataframe
DMA = rcurves[rcurves['TYPE'] == 'DMA']
DMR = rcurves[rcurves['TYPE'] == 'DMR']
DMO = rcurves[rcurves['TYPE'] == 'DMO']
DMC = rcurves[rcurves['TYPE'] == 'DMC']
t = pd.merge(DMA, DMR, on='DAYSSINCEDROP', how='outer')
x = pd.merge(DMC, DMO, on='DAYSSINCEDROP', how='outer')
rescurves = pd.merge(t, x, on='DAYSSINCEDROP', how='outer')
rescurves = rescurves.rename(columns={
    'DAYSSINCEDROP': 'DAYS_SINCE_DROP',
    'CURVE_x_x': 'DMA',
    'CURVE_y_x': 'DMR',
    'CURVE_x_y': 'DMC',
    'CURVE_y_y': 'DMO'
})
rescurves = rescurves[['DAYS_SINCE_DROP', 'DMA', 'DMR', 'DMC', 'DMO']]
rescurves = rescurves.fillna(0)
rescurves

,DAYS_SINCE_DROP,DMA,DMR,DMC,DMO
0,2,0,0,0,0.000152
1,3,0.007141,0.003328,0,0.002592
2,4,0.038017,0.028622,0.016902,0.026229
3,5,0.082359,0.065231,0.047887,0.063287
4,6,0.104189,0.047482,0.033803,0.039649
5,7,0.108610,0.120812,0.174648,0.133512
6,8,0.074878,0.137897,0.129577,0.160198
7,9,0.046722,0.109053,0.067606,0.112085
8,10,0.086303,0.075438,0.070423,0.074113
9,11,0.071613,0.053583,0.078874,0.057492


In [ ]:
# forecast dfs
forecast = pd.DataFrame()
date_range = pd.date_range(start=pd.to_datetime(start), end=pd.to_datetime(end)) # change for different date range
column_names = [date.strftime('%Y-%m-%d') for date in date_range]
for col in column_names:
    forecast[col] = np.nan
zeros_row = pd.DataFrame([[Decimal(0)] * len(forecast.columns)], columns=forecast.columns)
response_forecast = forecast
ss_sub_forecast = forecast
ib_sub_forecast = forecast
prelim_forecast = forecast
call_forecast = pd.concat([forecast, zeros_row], ignore_index=True)
response_sum = pd.concat([forecast, zeros_row], ignore_index=True)
ss_sub_sum = pd.concat([forecast, zeros_row], ignore_index=True)
sub_sum = pd.concat([forecast, zeros_row], ignore_index=True)
prelim_sum = pd.concat([forecast, zeros_row], ignore_index=True)

# forecasting function
for i in relevant_campaigns.index:
    response_forecast = pd.concat([response_forecast, zeros_row], ignore_index=True)
    ss_sub_forecast = pd.concat([ss_sub_forecast, zeros_row], ignore_index=True)
    ib_sub_forecast = pd.concat([ib_sub_forecast, zeros_row], ignore_index=True)
    prelim_forecast = pd.concat([prelim_forecast, zeros_row], ignore_index=True)
    # store campaign mail date
    temp_date = relevant_campaigns.loc[i, 'MAIL_DATE']
    # determine campaign type
    if relevant_campaigns.loc[i, 'TYPE'] == 'DMA':
        temp_type = 'DMA'
    elif relevant_campaigns.loc[i, 'TYPE'] == 'DMR':
        temp_type = 'DMR'
    elif relevant_campaigns.loc[i, 'TYPE'] == 'DMC':
        temp_type = 'DMC'
    elif relevant_campaigns.loc[i, 'TYPE'] == 'DMO':
        temp_type = 'DMO'
    # for every day after mail date, update response forecast with projections times curves
    for j in rescurves.index:
        cur_date = temp_date + relativedelta(days=int(rescurves.loc[j,'DAYS_SINCE_DROP']))
        cur_as_str = cur_date.strftime('%Y-%m-%d')
        if cur_as_str in forecast.columns:
            # responses
            amt_response_to_add = relevant_campaigns.loc[i, 'PROJECTED_RESPONSES'] * rescurves.loc[j, temp_type]
            response_forecast.at[i, cur_as_str] = response_forecast.at[i, cur_as_str] + amt_response_to_add
            response_sum.at[0, cur_as_str] = response_sum.at[0, cur_as_str] + amt_response_to_add
            # ss subs
            amt_ss_sub_to_add = amt_response_to_add * Decimal(str(relevant_campaigns.loc[i, 'SS_SUB_RATE']/100 * (1 - ib_rate.loc[0, temp_type])))
            ss_sub_forecast.at[i, cur_as_str] = ss_sub_forecast.at[i, cur_as_str] + (amt_ss_sub_to_add)
            sub_sum.at[0, cur_as_str] = sub_sum.at[0, cur_as_str] + amt_ss_sub_to_add
            ss_sub_sum.at[0, cur_as_str] = ss_sub_sum.at[0, cur_as_str] + amt_ss_sub_to_add
            # ib subs
            amt_ib_sub_to_add = amt_response_to_add * Decimal(str(relevant_campaigns.loc[i, 'IB_SUB_RATE']/100 * ib_rate.loc[0, temp_type]))
            ib_sub_forecast.at[i, cur_as_str] = ib_sub_forecast.at[i, cur_as_str] + amt_ib_sub_to_add
            sub_sum.at[0, cur_as_str] = sub_sum.at[0, cur_as_str] + amt_ib_sub_to_add
            # prelims
            amt_prelim_to_add = amt_ib_sub_to_add + amt_ss_sub_to_add * Decimal(str(relevant_campaigns.loc[i, 'PRELIM_RATE']/100))
            prelim_forecast.at[i, cur_as_str] = prelim_forecast.at[i, cur_as_str] + amt_prelim_to_add
            prelim_sum.at[0, cur_as_str] = prelim_sum.at[0, cur_as_str] + amt_prelim_to_add
            # ib calls
            amt_call_to_add = amt_ib_sub_to_add/call_to_sub
            call_forecast.at[0, cur_as_str] = call_forecast.at[0, cur_as_str] + amt_call_to_add

# twilio preapproved calls and twilio abandons, add to call forecast
prelim_rate = prelim_sum.iloc[0].sum()/sub_sum.iloc[0].sum()
holidays = pd.Series(['12/25/2024', '12/26/2024', '1/1/2024', '1/15/2024',
            '2/19/2024', '5/27/2024', '6/19/2024', '7/4/2024',
            '9/2/2024', '11/28/2024', '11/29/2024', '12/24/2024',
            '12/25/2024'])
holidays = pd.to_datetime(holidays, format='%m/%d/%Y')
month_day = holidays.dt.month.astype(str) + '-' + holidays.dt.day.astype(str)
call_forecast = pd.concat([call_forecast, zeros_row, zeros_row], ignore_index=True)
call_forecast = pd.concat([call_forecast, call_forecast.iloc[[0]]], ignore_index=True)

for i in forecast.columns:
    cur = datetime.strptime(i, '%Y-%m-%d')
    month_day_str = f'{cur.month}-{cur.day}'
    ab_call = (response_sum.at[0, i] - sub_sum.at[0, i]) * ab_ss * ss_bizhrs
    call_forecast.at[2, i] = call_forecast.at[2, i] + ab_call
    call_forecast.at[3, i] = call_forecast.at[3, i] + ab_call

    # # twilio during business hours
    # amt_preapp_call_to_add = ss_sub_sum.at[0, i] * twilio * prelim_rate
    # call_forecast.at[1, i] = call_forecast.at[1, i] + amt_preapp_call_to_add
    # call_forecast.at[3, i] = call_forecast.at[3, i] + amt_preapp_call_to_add

    # no twillio during business hours
    if (cur.weekday() not in [6]) & (month_day_str not in month_day.values):
        amt_preapp_call_to_add = ss_sub_sum.at[0, i] * twilio * prelim_rate
        call_forecast.at[1, i] = call_forecast.at[1, i] + amt_preapp_call_to_add
        call_forecast.at[3, i] = call_forecast.at[3, i] + amt_preapp_call_to_add

def quantize_decimal(value, decimal_places=2, rounding_mode=ROUND_CEILING):
    if pd.isna(value):
        return value
    decimal_value = Decimal(str(value))
    rounded_value = decimal_value.quantize(Decimal('1e-{0}'.format(decimal_places)), rounding=rounding_mode)
    return rounded_value

# round numbers and concat to corresponding campaigns (imitating louise sheet)
response_forecast = response_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
response_forecast = pd.concat([relevant_campaigns, response_forecast], axis=1)

ss_sub_forecast = ss_sub_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
ss_sub_forecast = pd.concat([relevant_campaigns, ss_sub_forecast], axis=1)

ib_sub_forecast = ib_sub_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
ib_sub_forecast = pd.concat([relevant_campaigns, ib_sub_forecast], axis=1)

prelim_forecast = prelim_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
prelim_forecast = pd.concat([relevant_campaigns, prelim_forecast], axis=1)

call_forecast = call_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
call_forecast = call_forecast.rename(index={
    0: 'inbound',
    1: 'twilio preapproved',
    2: 'twilio abandons',
    3: 'total'
})
call_forecast

2024-07-26 00:00:00 0.832524
2024-07-29 00:00:00 0.693360
2024-07-29 00:00:00 0.181376
2024-08-02 00:00:00 0.763776
2024-08-05 00:00:00 0.343674
2024-08-05 00:00:00 1.265584
2024-08-05 00:00:00 0.542464
2024-08-09 00:00:00 0.721344
2024-08-12 00:00:00 1.809216
2024-08-12 00:00:00 0.864114
2024-08-16 00:00:00 1.562912
2024-08-19 00:00:00 0.033804
2024-08-19 00:00:00 1.780689
2024-08-19 00:00:00 0.898776
2024-08-23 00:00:00 2.413658
2024-08-26 00:00:00 2.792273
2024-08-26 00:00:00 3.555948
2024-08-30 00:00:00 4.692842
2024-09-06 00:00:00 21.307650
2024-09-09 00:00:00 1.718309
2024-09-09 00:00:00 21.967938
2024-09-09 00:00:00 4.147568
2024-09-13 00:00:00 37.238760
2024-09-16 00:00:00 131.842954
2024-09-16 00:00:00 21.236138
2024-09-20 00:00:00 19.768840


C:\Users\Vance.Liu\AppData\Local\Temp\ipykernel_16768\1509618520.py:100: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  response_forecast = response_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
C:\Users\Vance.Liu\AppData\Local\Temp\ipykernel_16768\1509618520.py:103: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ss_sub_forecast = ss_sub_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
C:\Users\Vance.Liu\AppData\Local\Temp\ipykernel_16768\1509618520.py:106: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ib_sub_forecast = ib_sub_forecast.applymap(lambda x: quantize_decimal(x, decimal_places=0, rounding_mode=ROUND_CEILING))
C:\Users\Vance.Liu\AppData\Local\Temp\ipykernel_16768\1509618520.py:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  preli

,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08,2024-09-09,2024-09-10,...,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28,2024-09-29,2024-09-30
inbound,26,56,62,54,47,41,27,18,35,42,...,51,33,79,88,72,61,61,55,36,86
twilio preapproved,0,0,32,27,23,20,13,0,17,21,...,28,0,44,49,40,33,33,31,0,49
twilio abandons,19,40,45,40,35,31,21,14,26,32,...,28,19,41,45,41,37,36,31,21,46
total,45,96,138,121,104,91,60,31,77,94,...,106,52,164,181,152,130,129,116,57,180


In [ ]:
call_forecast.to_csv("output/call_forecast_9-24.csv")

In [ ]:
# update response rate table
add = (new_camps.drop(['MAIL_DATE', 'QUANTITY'], axis=1)).drop_duplicates()
add['MONTH'] = next.month
t = response_rates
unique_rows = t.merge(add, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
unique_rows.to_csv('ResponseRateInput.csv')